In [1]:
# Necessary for building architecture
from keras import models
from keras import layers
from keras import backend as K

# Necessary for classification
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

# Necessary to dataset preprocessing
from keras.preprocessing.image import img_to_array
import cv2
# from imagelab.preprocessing import ImageToArrayPreprocessor
# from imagelab.preprocessing import ImageResizePreprocessor
# from imagelab.datasets import DatasetLoader


Using TensorFlow backend.


# ImageLab Preprocessing Modules

In [2]:
# Image To Array Preprocessor
class ImageToArrayPreprocessor:
    # Constructor
    def __init__(self, dataFormat=None):
        # Store the data format
        self.dataFormat = dataFormat
        
    # Data format processor
    def process(self, image):
        # Process data using keras img_to_array method
        return img_to_array(image, data_format=self.dataFormat)

In [3]:
# Image Resize Preprocessor
class ImageResizePreprocessor:
    # Constructor
    def __init__(self, width, height, inter=cv2.INTER_AREA):
        # Store the target image width, height and interpolation method used when resizing
        self.width = width
        self.height = height
        self.inter = inter
    
    # Size Processor
    def process(self, image):
        # Resize image while ignoring the aspect ratio
        return cv2.resize(image, (self.width, self.height), interpolation=self.inter)

# ImageLab Datasets Module

In [4]:
# Dataset Loader
class DatasetLoader:
    # Constructor
    def __init__(self, preprocessors=None):
        # Store the image preprocessor
        self.preprocessors = preprocessors
        
        # Initialize preprocessors as a list of its None
        if self.preprocessors is None:
            self.preprocessors = []
    
    # Loader
    def load(self, imagePaths, verbose=1):
        # Initialize the list of features and labels
        data = []
        labels = []
        
        # Loop over the input image
        for (i, imagePath) in enumerate(imagePaths):
            # Load image and extract the class labels
            image = cv2.imread(imagePath)
            image_label = imagePath.split(os.path.sep)[-1]
            label = image_label[:3]
            
            # Check if preprocessor is not None
            if self.preprocessors is not None:
                # Loop over the preprocessors and apply each to the image
                for p in self.preprocessors:
                    image = p.process(image)
            
            # Treat preprocessed image as a feature vector by updating the data list and the labels
            data.append(image)
            labels.append(label)
            
            # Show an update every verbose image
            if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
                print('[INFO] processed {}/{}'.format(i + 1, len(imagePaths)))
        
        # Return a tuple of the data and labels
        return (np.array(data), np.array(labels))

# Model Architecture

In [5]:
class shallowNet:
    @staticmethod
    def build(width, height, depth, classes):
        # initialize the model and input shape to be channel_last
        model = models.Sequential()
        input_shape = (width, height, depth)
        
        # if we are using channel_first, update the input_shape accordingly
        if K.image_data_format() == 'channels_first':
            input_shape = (depth, width, height)
        
        # Define a models
        model.add(layers.Conv2D(32, (3, 3), padding='same', input_shape=input_shape))
        model.add(layers.Activation('relu'))
        model.add(layers.Flatten())
        model.add(layers.Dense(1, activation='softmax'))

        
        # return constructed network
        return model

# Build a model that accept (32 x 32 x3) rgb image organized in 10 classes
model = shallowNet.build(32, 32, 3, 2)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 32769     
Total params: 33,665
Trainable params: 33,665
Non-trainable params: 0
_________________________________________________________________


# Data Loading and Processing

In [6]:
base_dir = 'C:/Users/Tajr/Desktop/Data/RadonPlus/RadonTechnology/Dev/Deep Learning/Datasets/CatDog/SmallDataset'

# Loading Training Cats
train_cats_base_dir = os.path.join(base_dir, 'train/cats')
train_cats_images = list(paths.list_images(train_cats_base_dir))

# Loading Training Dogs
train_dogs_base_dir = os.path.join(base_dir, 'train/dogs')
train_dogs_images = list(paths.list_images(train_dogs_base_dir))

# Group Training Cats and Dogs
train_images_path = train_cats_images

for path in train_dogs_images:
    train_images_path.append(path)

# Loading Validation Cats
validation_cats_base_dir = os.path.join(base_dir, 'validation/cats')
validation_cats_images = list(paths.list_images(validation_cats_base_dir))

# Loading Validation Dogs
validation_dogs_base_dir = os.path.join(base_dir, 'validation/dogs')
validation_dogs_images = list(paths.list_images(validation_dogs_base_dir))

# Group Validation Cats and Dogs
validation_images_path = validation_cats_images

for path in validation_dogs_images:
    validation_images_path.append(path)

# Loading Test Cats
test_cats_base_dir = os.path.join(base_dir, 'test/cats')
test_cats_images = list(paths.list_images(test_cats_base_dir))

# Loading Test Dogs
test_dogs_base_dir = os.path.join(base_dir, 'test/dogs')
test_dogs_images = list(paths.list_images(test_dogs_base_dir))

# Grouping Test Cats and Dogs
test_images_path = test_cats_images

for path in test_dogs_images:
    test_images_path.append(path)

In [7]:
print("First Train Image Path")
train_images_path[0]

First Train Image Path


'C:/Users/Tajr/Desktop/Data/RadonPlus/RadonTechnology/Dev/Deep Learning/Datasets/CatDog/SmallDataset\\train/cats\\cat.0.jpg'

In [8]:
print("First Validation Image Path")
validation_images_path[0]

First Validation Image Path


'C:/Users/Tajr/Desktop/Data/RadonPlus/RadonTechnology/Dev/Deep Learning/Datasets/CatDog/SmallDataset\\validation/cats\\cat.1000.jpg'

In [9]:
print("First Test Image Path")
test_images_path[0]

First Test Image Path


'C:/Users/Tajr/Desktop/Data/RadonPlus/RadonTechnology/Dev/Deep Learning/Datasets/CatDog/SmallDataset\\test/cats\\cat.1500.jpg'

In [10]:
train_image_label = train_images_path[1999].split(os.path.sep)[-1]
print(train_image_label)

dog.999.jpg


In [11]:
train_label = train_image_label[:3]
print(train_label)

dog


In [12]:
# # Data Preprocessing

# Initialize the Image Preprocessor
irp = ImageResizePreprocessor(32, 32)
i2a = ImageToArrayPreprocessor()

# Label Encoder
lb = LabelBinarizer()

# Load Data and apply image the scale the pixels intensities to the range [0, 1]
dl = DatasetLoader(preprocessors=[irp, i2a])

In [13]:
# Train Data and Labels
(train_data, train_labels) = dl.load(train_images_path, verbose=500)
train_data = train_data.astype('float') / 255.0
train_labels = lb.fit_transform(train_labels)


[INFO] processed 500/2000
[INFO] processed 1000/2000
[INFO] processed 1500/2000
[INFO] processed 2000/2000


In [14]:
# Validation Data and Labels
(validation_data, validation_labels) = dl.load(validation_images_path, verbose=500)
validation_data.astype('float') / 255.0
validation_labels = lb.fit_transform(validation_labels)

[INFO] processed 500/1000
[INFO] processed 1000/1000


In [15]:
# Test Data and Labels
(test_data, test_labels) = dl.load(test_images_path, verbose=500)
test_data.astype('float') / 255.0
test_labels = lb.fit_transform(test_labels)


[INFO] processed 500/1000
[INFO] processed 1000/1000


# Compile and Train a Model

In [17]:
sgd = SGD(lr=0.01)
model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['acc'])
history = model.fit(train_data, train_labels, epochs=100, batch_size=32, verbose=1, validation_data=(validation_data, validation_labels))

Train on 2000 samples, validate on 1000 samples
Epoch 1/100
2000/2000 [==============================] - 1s 400us/step - loss: 7.6666 - acc: 0.5000 - val_loss: 7.6246 - val_acc: 0.5000
Epoch 2/100
2000/2000 [==============================] - 1s 302us/step - loss: 7.6666 - acc: 0.5000 - val_loss: 7.6246 - val_acc: 0.5000
Epoch 3/100
2000/2000 [==============================] - 1s 309us/step - loss: 7.6666 - acc: 0.5000 - val_loss: 7.6246 - val_acc: 0.5000
Epoch 4/100
2000/2000 [==============================] - 1s 301us/step - loss: 7.6666 - acc: 0.5000 - val_loss: 7.6246 - val_acc: 0.5000
Epoch 5/100
2000/2000 [==============================] - 1s 305us/step - loss: 7.6666 - acc: 0.5000 - val_loss: 7.6246 - val_acc: 0.5000
Epoch 6/100
2000/2000 [==============================] - 1s 310us/step - loss: 7.6666 - acc: 0.5000 - val_loss: 7.6246 - val_acc: 0.5000
Epoch 7/100
2000/2000 [==============================] - 1s 309us/step - loss: 7.6666 - acc: 0.5000 - val_loss: 7.6246 - val_acc: 

KeyboardInterrupt: 

# Plotting Results

# Model Evaluation